In [1]:
import os
import sys
module_path = os.path.abspath(os.path.join('..'))
if module_path not in sys.path:
    sys.path.append(module_path)

In [2]:
import os
import django

os.environ['DJANGO_SETTINGS_MODULE']='TweetToEmoji.settings'

django.setup()

In [158]:
from app.models import Emoji
from nltk.corpus import wordnet 
from nltk import pos_tag
from nltk.tokenize import sent_tokenize

In [4]:
def TweetsFromResponse(response):
    return [t.get('text') for t in response.get('statuses')]
        

In [135]:
from django.db.models import Q
def SearchEmoji(keyword):
    emoji = Emoji.objects.filter(Q(Name__contains = keyword) | Q(Description__contains = keyword))
    return ''.join([e.Icon+str(e.Id)+e.Name for e in emoji])

In [130]:
def FindSynonyms(word):
    syns = wordnet.synsets(word) 
    synonym_list =[s.lemma_names() for s in syns ] # take synonyms name
    synonym_list = [item.replace('_',' ') for sublist in synonym_list for item in sublist] # flatten list
    synonym_list= set(synonym_list) # remove duplicates
    return list(synonym_list)

In [1]:
from TwitterService.Client import TwitterClient

client = TwitterClient()

In [8]:
content = client.Get('search/tweets.json', lang = 'en', q='climate', result_type = 'popular')

In [161]:

content = client.GetFromHistory("04-11-2018 17.06.12.391")

In [9]:
tweets = TweetsFromResponse(content)
tweets

['People are going to die if we don’t start addressing climate change ASAP.\n\nIt’s not enough to think it’s “important… https://t.co/fzfg6ppv1y',
 'Since this was released on a day no one would notice it... as in a Friday during a holiday weekend..let’s make sure… https://t.co/P71S5PDLfZ',
 'A massive federal climate change report that was due next month is out today. It warns of devastating health and ec… https://t.co/nT1pqtrsW4',
 'Try as the Trump Administration might to bury this study on the Friday after Thanksgiving, the effects of climate c… https://t.co/GhoDU4ewrg',
 'The Trump administration tried to bury a new report about the devastating consequences of climate change. \n\nWhy? Be… https://t.co/tJCcNtVlVi',
 'For your records, @realDonaldTrump. \n\nYOUR government’s report on climate report. \n\nYou should really read this. It… https://t.co/N9WljPRlGV',
 'The climate crisis threatens both our communities &amp; our economy. While @realDonaldTrump ignores his own admin’s rep…

In [101]:
FindSynonyms("burns")

{'Burns',
 'George Burns',
 'Nathan Birnbaum',
 'Robert Burns',
 'bite',
 'burn',
 'burn down',
 'burn mark',
 'burn off',
 'burn up',
 'burning',
 'cauterise',
 'cauterize',
 'combust',
 'cut',
 'fire',
 'glow',
 'incinerate',
 'sting',
 'sunburn',
 'suntan',
 'tan'}

In [151]:
def TagOf(word):
    return pos_tag([s])[0][1]

In [157]:
word="dogs"
syn = FindSynonyms(word)
print(word +' is ' + TagOf(word))
for s in syn:
    print('{0}({1}) -> {2}'.format(s, TagOf(s), SearchEmoji(s)))

dogs is NN
track(NN) -> 🌸1729 Cherry Blossom🚈1926 Light Rail🚝1929 Monorail🛤1954 Railway Track🚟1976 Suspension Railway🖲2171 Trackball🔖2207 Bookmark🔀2390 Shuffle Tracks Button🔁2391 Repeat Button🔂2392 Repeat Single Button⏭2395 Next Track Button⏮2399 Last Track Button
wienerwurst(NN) -> 
weenie(NN) -> 
blackguard(NN) -> 
give chase(NN) -> 
frank(NN) -> 
tag(NN) -> 🌷1737 Tulip🛑1960 Stop Sign⚡2052 High Voltage#2448 Keycap Number Sign🔣2465 Input Symbols
click(NN) -> 
dog(NN) -> 🥺90 Pleading Face🙇844 Person Bowing🦴1554 Bone🐶1629 Dog Face🐕1630 Dog🐩1631 Poodle🐾1678 Paw Prints🌭1793 Hot Dog
heel(NN) -> 🤸1220 Person Cartwheeling🤸1226 Man Cartwheeling🤸1232 Woman Cartwheeling🥿1612 Flat Shoe👠1613 High-Heeled Shoe👡1614 Woman’s Sandal👢1615 Woman’s Boot🍭1835 Lollipop🎡1916 Ferris Wheel🛴1949 Kick Scooter🧳1983 Luggage🎰2119 Slot Machine♿2320 Wheelchair Symbol☸2369 Wheel of Dharma
andiron(NN) -> 
hound(NN) -> 
pawl(NN) -> 
frankfurter(NN) -> 
tail(NN) -> 👧142 Girl👩160 Woman🐒1627 Monkey🐩1631 Poodle🐈1636 Cat🐿16

In [142]:
import nltk
nltk.download('averaged_perceptron_tagger')

[nltk_data] Downloading package averaged_perceptron_tagger to
[nltk_data]     C:\Users\leven\AppData\Roaming\nltk_data...
[nltk_data]   Unzipping taggers\averaged_perceptron_tagger.zip.


True

In [219]:
def GenerateCombinations(words):
    list = []
    for slen in range(1, len(words)):
        for sshift in range(0, len(words) - slen):
            list.append(words[sshift:slen + sshift])
    return list

In [209]:
def Tokenize(sentence):
    tokenized_text = nltk.word_tokenize(sentence)
    return nltk.pos_tag(tokenized_text)
        

In [229]:
def ConvertTweet(tweet):
    sentences = sent_tokenize(tweet)
    words=Tokenize(sentences[0])
    combinations = GenerateCombinations(words)
    combinations = sorted(combinations, key=len, reverse=True)
    print(combinations)

In [230]:
ConvertTweet(tweets[0])

[[('Do', 'VBP'), ('you', 'PRP'), ('like', 'IN'), ('blowing', 'VBG'), ('bubbles', 'NNS')], [('Do', 'VBP'), ('you', 'PRP'), ('like', 'IN'), ('blowing', 'VBG')], [('you', 'PRP'), ('like', 'IN'), ('blowing', 'VBG'), ('bubbles', 'NNS')], [('Do', 'VBP'), ('you', 'PRP'), ('like', 'IN')], [('you', 'PRP'), ('like', 'IN'), ('blowing', 'VBG')], [('like', 'IN'), ('blowing', 'VBG'), ('bubbles', 'NNS')], [('Do', 'VBP'), ('you', 'PRP')], [('you', 'PRP'), ('like', 'IN')], [('like', 'IN'), ('blowing', 'VBG')], [('blowing', 'VBG'), ('bubbles', 'NNS')], [('Do', 'VBP')], [('you', 'PRP')], [('like', 'IN')], [('blowing', 'VBG')], [('bubbles', 'NNS')]]
